# Word2Vec Tutorial

In case you missed the buzz, word2vec is a widely featured as a member of the “new wave” of machine learning algorithms based on neural networks, commonly referred to as "deep learning" (though word2vec itself is rather shallow). Using large amounts of unannotated plain text, word2vec learns relationships between words automatically. The output are vectors, one vector per word, with remarkable linear relationships that allow us to do things like vec(“king”) – vec(“man”) + vec(“woman”) =~ vec(“queen”), or vec(“Montreal Canadiens”) – vec(“Montreal”) + vec(“Toronto”) resembles the vector for “Toronto Maple Leafs”.

Word2vec is very useful in [automatic text tagging](https://github.com/RaRe-Technologies/movie-plots-by-genre), recommender systems and machine translation.

Check out an [online word2vec demo](http://radimrehurek.com/2014/02/word2vec-tutorial/#app) where you can try this vector algebra for yourself. That demo runs `word2vec` on the Google News dataset, of **about 100 billion words**.

## This tutorial

In this tutorial you will learn how to train and evaluate word2vec models on your business data.  
https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/word2vec.ipynb


## Preparing the Input
Starting from the beginning, gensim’s `word2vec` expects a sequence of sentences as its input. Each sentence is a list of words (utf8 strings):

In [1]:
#Ensure gensim  and Cython are installed
#!pip list

In [2]:
# import modules & set up logging
import os

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import gensim

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Training Word2vec on Three in a Boat book

In [3]:
directory = 'C://Users//IBM_ADMIN//Documents//Teaching//Data Projects//Text//'
book = 'Books//3boat10.txt'

f = open(directory+book, encoding="utf8")
text = f.read()

In [4]:
class MyText(object):
    def __iter__(self):
        for line in open(directory+book, encoding="utf8"):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

sentences = MyText()

print(sentences)

## Training
`Word2Vec` accepts several parameters that affect both training speed and quality.

### min_count
`min_count` is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them:

In [5]:
# default value of min_count=5
model = gensim.models.Word2Vec(sentences, min_count=10)

### size
`size` is the number of dimensions (N) of the N-dimensional space that gensim Word2Vec maps the words onto.

Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

In [6]:
# default value of size=100
model = gensim.models.Word2Vec(sentences, size=100)

### workers
`workers`, the last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) is for training parallelization, to speed up training:

In [7]:
# default value of workers=3
model = gensim.models.Word2Vec(sentences, workers=4)

The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `word2vec` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).

## Memory
At its core, `word2vec` model parameters are stored as matrices (NumPy arrays). Each array is **#vocabulary** (controlled by min_count parameter) times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`Word2Vec` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. It is provided in the 'datasets' folder.

For example a syntactic analogy of comparative type is bad:worse;good:?. There are total of 9 types of syntactic comparisons in the dataset like plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as capital cities (Paris:France;Tokyo:?) or family members (brother:sister;dad:?). 

Gensim supports the same evaluation set, in exactly the same format:

In [8]:
directory_w2v = 'file://C://Users//IBM_ADMIN//Documents//Teaching//Data Projects//Text//w2v//'

In [9]:
model.wv.accuracy(directory_w2v + 'questions-words.txt')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `accuracy` (Method will be removed in 4.0.0, use self.evaluate_word_analogies() instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[{'section': 'capital-common-countries', 'correct': [], 'incorrect': []},
 {'section': 'capital-world', 'correct': [], 'incorrect': []},
 {'section': 'currency', 'correct': [], 'incorrect': []},
 {'section': 'city-in-state', 'correct': [], 'incorrect': []},
 {'section': 'family',
  'correct': [('HIS', 'HER', 'HE', 'SHE')],
  'incorrect': [('BOY', 'GIRL', 'FATHER', 'MOTHER'),
   ('BOY', 'GIRL', 'HE', 'SHE'),
   ('BOY', 'GIRL', 'HIS', 'HER'),
   ('BOY', 'GIRL', 'MAN', 'WOMAN'),
   ('FATHER', 'MOTHER', 'HE', 'SHE'),
   ('FATHER', 'MOTHER', 'HIS', 'HER'),
   ('FATHER', 'MOTHER', 'MAN', 'WOMAN'),
   ('FATHER', 'MOTHER', 'BOY', 'GIRL'),
   ('HE', 'SHE', 'HIS', 'HER'),
   ('HE', 'SHE', 'MAN', 'WOMAN'),
   ('HE', 'SHE', 'BOY', 'GIRL'),
   ('HE', 'SHE', 'FATHER', 'MOTHER'),
   ('HIS', 'HER', 'MAN', 'WOMAN'),
   ('HIS', 'HER', 'BOY', 'GIRL'),
   ('HIS', 'HER', 'FATHER', 'MOTHER'),
   ('MAN', 'WOMAN', 'BOY', 'GIRL'),
   ('MAN', 'WOMAN', 'FATHER', 'MOTHER'),
   ('MAN', 'WOMAN', 'HE', 'SHE'),
   ('

This `accuracy` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.



In the December 2016 release of Gensim we added a better way to evaluate semantic similarity.

By default it uses an academic dataset WS-353 but one can create a dataset specific to your business based on it. It contains word pairs together with human-assigned similarity judgments. It measures the relatedness or co-occurrence of two words. For example, 'coast' and 'shore' are very similar as they appear in the same context. At the same time 'clothes' and 'closet' are less similar because they are related but not interchangeable.

In [10]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep

In [11]:
model.wv.evaluate_word_pairs(test_data_dir + 'wordsim353.tsv')

((0.351962279120603, 0.3185780936637547),
 SpearmanrResult(correlation=0.41818181818181815, pvalue=0.22911284098281892),
 97.16713881019831)

Once again, **good performance on Google's or WS-353 test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task. For an example of how to use word2vec in a classifier pipeline, see this [tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre).

## Storing and loading models
You can store/load models using the standard gensim methods:

In [12]:
directory_w2v_model = directory_w2v + "book_model"
directory_w2v_model

'file://C://Users//IBM_ADMIN//Documents//Teaching//Data Projects//Text//w2v//book_model'

In [13]:
model.save(directory_w2v_model)  # save the model

In [14]:
new_model = gensim.models.Word2Vec.load(directory_w2v_model)  # open the model

which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.

In addition, you can load models created by the original C tool, both using its text and binary formats:
```
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
  # using gzipped/bz2 input works too, no need to unzip:
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)
```

## Online training / Resuming training
Advanced users can load a model and continue training it with more sentences and [new vocabulary words](online_w2v_tutorial.ipynb):

In [15]:
model = gensim.models.Word2Vec.load(directory_w2v_model)
more_sentences = [['Advanced', 'users', 'can', 'load', 'a', 'model', 'and', 'continue', 'training', 'it', 'with', 'more', 'sentences']]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.epochs)

# cleaning up temp
# os.close(fs)
# os.remove(temp_path)

(13, 65)

You may need to tweak the `total_words` parameter to `train()`, depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C tool, `KeyedVectors.load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.

## Using the model
`Word2Vec` supports several word similarity tasks out of the box:

In [16]:
if 'tea' in model:
    print(model['tea'].shape)
else:
    print('{0} is an out of dictionary word'.format('tea'))

(100,)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [17]:
model.wv.most_similar(positive=['water', 'river'], negative=['tea'], topn=1)

[('and,', 0.9991182088851929)]

In [18]:
print(model.wv.similarity('head', 'boat'))
print(model.wv.similarity('river', 'water'))

0.99978054
0.99985576


You can get the probability distribution for the center word given the context words as input:

In [19]:
print(model.predict_output_word(['boat', 'river', 'water']))

[('can', 0.0017630099), ('more', 0.0016664715), ('it,', 0.0013456458), ('been', 0.0012891365), ('people', 0.0012842724), ('come', 0.0012729266), ('got', 0.0012689896), ('never', 0.0012579266), ('them', 0.0012566646), ('it.', 0.0012539034)]


The results here don't look good because the training corpus is very small. To get meaningful results one needs to train on 500k+ words.

If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [20]:
model.wv.__getitem__('boat')  # raw NumPy vector of a word

array([-0.3942015 ,  0.23584281, -0.04920526,  0.11957395, -0.35226712,
       -0.19367567, -0.27025673,  0.2104711 , -0.15261875, -0.08015857,
       -0.17626545,  0.4151441 , -0.5112834 ,  0.2389548 , -0.4497126 ,
        0.5928972 , -0.09446308,  0.41125774, -0.03768824,  0.45991328,
        0.11137432,  0.0313634 , -0.14867884,  0.49344093,  0.45399904,
        0.49134538,  0.5356065 ,  0.27955425, -0.20928909,  0.4018742 ,
       -0.5052954 ,  0.47832105, -0.12731785,  0.04730815, -0.44345796,
       -0.26815686,  0.45327646, -0.5460642 ,  0.5275853 ,  0.45769358,
        0.09576337,  0.04278165, -0.21387278, -0.01210997, -0.17602871,
        0.5329506 ,  0.17196184, -0.0592141 , -0.25488833, -0.51073647,
       -0.14272785,  0.3889757 , -0.31409946, -0.4075012 ,  0.02974972,
       -0.61988455,  0.6481276 ,  0.21580404,  0.31985068, -0.15068793,
       -0.45306465,  0.11432141, -0.01921608,  0.12702253, -0.14964059,
        0.02991934, -0.17144321, -0.17028883,  0.16534679, -0.59

…or en-masse as a 2D NumPy matrix from `model.wv.syn0`.

## Training Loss Computation

The parameter `compute_loss` can be used to toggle computation of loss while training the Word2Vec model. The computed loss is stored in the model attribute `running_training_loss` and can be retrieved using the function `get_latest_training_loss` as follows : 

In [21]:
# instantiating and training the Word2Vec model
model_with_loss = gensim.models.Word2Vec(sentences, min_count=1, compute_loss=True, hs=0, sg=1, seed=42)

# getting the training loss value
training_loss = model_with_loss.get_latest_training_loss()
print(training_loss)

3566317.5078075198


## Conclusion

In this tutorial we learned how to train word2vec models on your custom data and also how to evaluate it. Hope that you too will find this popular tool useful in your Machine Learning tasks!

## Links


Full `word2vec` API docs [here](http://radimrehurek.com/gensim/models/word2vec.html); get [gensim](http://radimrehurek.com/gensim/) here. Original C toolkit and `word2vec` papers by Google [here](https://code.google.com/archive/p/word2vec/).